# TSIS 9 - PyGame

Это **#9** лабораторная работа по предмету *"Принципы Программирования 2".*

Студента: **Айбасов Даниал Елнарович**

**1. Racer**
Extend example project from Lab 8 and add following tasks: Extra tasks to the given tutorial:

- Randomly generating coins with different weights on the road
- Increase the speed of Enemy when the player earns N coins
- Comment your code

**2. Snake**
Extend example project from Lab 8 and add following tasks:

- Randomly generating food with different weights
- Foods which are disappearing after some time(timer)
- Comment your code

**3. Paint**
Extend example project from Lab 8 and add following tasks:

- Draw square
- Draw right triangle
- Draw equilateral triangle
- Draw rhombus
- Comment your code

In [2]:
import pygame
import sys
import random
import time
from pygame.locals import *

---

# 1. Racer

In [7]:
# Initializing
pygame.init()
pygame.mixer.init()  # Initialize the mixer for sound playback

# Setting up FPS 
FPS = 60
FramePerSec = pygame.time.Clock()

# Colors
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Screen Dimensions and Game Variables
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0
COIN_SCORE = 0
coin_number = 0

# Fonts and Texts
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)

# Background Image
background = pygame.image.load("assets/AnimatedStreet.png")
DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Game")

# Coin Classes
class Coin(pygame.sprite.Sprite):
    def __init__(self, image_path, size, score_value):
        super().__init__()
        self.image = pygame.image.load(image_path)
        self.image = pygame.transform.scale(self.image, size)
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)
        self.score_value = score_value

    def move(self):
        self.rect.move_ip(0, 10)
        if self.rect.top > SCREEN_HEIGHT:
            self.reset_position()

    def reset_position(self):
        self.rect.top = 0
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

# Specific Coin Instances
coin25 = Coin("assets/coin_25.png", (30, 30), 25)
coin50 = Coin("assets/coin_50.png", (40, 40), 50)
coin75 = Coin("assets/coin_75.png", (50, 50), 75)

# Enemy Class
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("assets/Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            SCORE += 1
            self.reset_position()

    def reset_position(self):
        self.rect.top = 0
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

# Player Class
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("assets/Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if pressed_keys[K_LEFT] and self.rect.left > 0:
            self.rect.move_ip(-5, 0)
        if pressed_keys[K_RIGHT] and self.rect.right < SCREEN_WIDTH:
            self.rect.move_ip(5, 0)

# Initialize Sprites
P1 = Player()
E1 = Enemy()
coins = [coin25, coin50, coin75]

# Sprite Groups
enemies = pygame.sprite.Group(E1)
all_sprites = pygame.sprite.Group(P1, E1, *coins)
coin_group = pygame.sprite.Group(*coins)

# Game Loop
while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    # Screen Update
    DISPLAYSURF.blit(background, (0, 0))
    scores = font_small.render(str(SCORE), True, BLACK)
    coin_scores = font_small.render('coins: ' + str(COIN_SCORE), True, BLACK)
    DISPLAYSURF.blit(scores, (10, 10))
    DISPLAYSURF.blit(coin_scores, (285, 10))

    # Move and Redraw Sprites
    for entity in all_sprites:
        DISPLAYSURF.blit(entity.image, entity.rect)
        entity.move()

    # Collision Detection with Coins
    for coin in coins:
        if pygame.sprite.collide_rect(P1, coin):
            COIN_SCORE += coin.score_value
            coin.reset_position()
            coin_number += 1
            if coin_number % 5 == 0:
                SPEED += 1

    # Collision Detection with Enemy
    if pygame.sprite.spritecollideany(P1, enemies):
        pygame.mixer.Sound('assets/crash.wav').play()
        time.sleep(0.5)
        DISPLAYSURF.fill(RED)
        DISPLAYSURF.blit(game_over, (30, 250))
        pygame.display.update()
        time.sleep(2)
        pygame.quit()
        sys.exit()

    pygame.display.update()
    FramePerSec.tick(FPS)


SystemExit: 

---

# 2. Snake

In [1]:
pygame.init()
WIDTH, HEIGHT = 800, 800
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
RED = (255, 0, 0)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
MAGENTA = (255, 0, 255)
FORESTGREEN = (34, 139, 34)
BLOCK_SIZE = 40
WHITE = (255, 255, 255)
SPEED = 5
SCORE = 0
level = 1
clock = pygame.time.Clock()

font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)
win = font.render("Congratulation", True, BLACK)

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class Snake:
    def __init__(self):
        self.body = [
            Point(
                x=WIDTH // BLOCK_SIZE // 2,
                y=HEIGHT // BLOCK_SIZE // 2,
            ),
            Point(
                x=WIDTH // BLOCK_SIZE // 2 + 1,
                y=HEIGHT // BLOCK_SIZE // 2,
            ),
        ]

    def draw(self):
        head = self.body[0]
        pygame.draw.rect(
            SCREEN,
            RED,
            pygame.Rect(
                head.x * BLOCK_SIZE,
                head.y * BLOCK_SIZE,
                BLOCK_SIZE,
                BLOCK_SIZE,
            )
        )
        for body in self.body[1:]:
            pygame.draw.rect(
                SCREEN,
                BLUE,
                pygame.Rect(
                    body.x * BLOCK_SIZE,
                    body.y * BLOCK_SIZE,
                    BLOCK_SIZE,
                    BLOCK_SIZE,
                )
            )

    def move(self, dx, dy):
        for idx in range(len(self.body) - 1, 0, -1):
            self.body[idx].x = self.body[idx - 1].x
            self.body[idx].y = self.body[idx - 1].y
        self.body[0].x += dx
        self.body[0].y += dy

        if self.body[0].x > WIDTH // BLOCK_SIZE:
            self.body[0].x = 0
        elif self.body[0].x < 0:
            self.body[0].x = WIDTH // BLOCK_SIZE
        elif self.body[0].y < 0:
            self.body[0].y = WIDTH // BLOCK_SIZE
        elif self.body[0].y > HEIGHT // BLOCK_SIZE:
            self.body[0].y = 0

    def check_collision(self, food):
        if food.location.x != self.body[0].x:
            return False
        if food.location.y != self.body[0].y:
            return False
        return True
    
    def check_collision_wall(self, wall):
        for i in range(0, len(wall.walls)):
            if wall.walls[i].x == self.body[0].x and wall.walls[i].y == self.body[0].y:
                return True
        return False


def draw_grid():
    for x in range(0, WIDTH, BLOCK_SIZE):
        pygame.draw.line(SCREEN, WHITE, start_pos=(x, 0), end_pos=(x, HEIGHT), width=1)
    for y in range(0, HEIGHT, BLOCK_SIZE):
        pygame.draw.line(SCREEN, WHITE, start_pos=(0, y), end_pos=(WIDTH, y), width=1)

class Food:
    def __init__(self, x, y):
        self.location = Point(x, y)
        self.color = GREEN

    def draw(self):
        pygame.draw.rect(
            SCREEN,
            self.color,
            pygame.Rect(
                self.location.x * BLOCK_SIZE,
                self.location.y * BLOCK_SIZE,
                BLOCK_SIZE,
                BLOCK_SIZE,
            )
        )
class Wall():
    def __init__(self, x_list, y_list):
        self.walls = []
        for i in range(0, len(x_list)):
            self.walls.append( Point(x_list[i],y_list[i]))

    def draw(self): 
        for wall in self.walls:
            pygame.draw.rect(
                SCREEN,
                MAGENTA,
                pygame.Rect(
                    wall.x * BLOCK_SIZE,
                    wall.y * BLOCK_SIZE,
                    BLOCK_SIZE,
                    BLOCK_SIZE,
                )
            )


def main():
    global SCORE
    global SPEED
    global level
    running = True
    snake = Snake()
    food = Food(5, 5)
    x1_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19 ] 
    y1_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
    x2_list = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    y2_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
    x3_list = [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]
    y3_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
    level1 = Wall( x1_list, y1_list)
    level2 = Wall( x2_list, y2_list)
    level3 = Wall( x3_list, y3_list)
    dx, dy = 0, 0
    FOOD_SPAWNER = pygame.USEREVENT + 1
    pygame.time.set_timer(FOOD_SPAWNER, 8000)

    while running:
        SCREEN.fill(BLACK)
        
        if ( SCORE == 0): 
            current_level = level1
        if ( SCORE == 4): 
            current_level = level2
            SPEED = 6
            level = 2
            while ( len( snake.body ) < 3):
                snake.body.pop()
        if ( SCORE == 8):
            current_level = level3
            SPEED = 8
            level = 3
        if ( SCORE == 12):
            current_level = level3
            SPEED = 10
            level = 4
            
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            if event.type == FOOD_SPAWNER:
                condition = True
                while condition:
                    current_color = random.choice( [GREEN, FORESTGREEN] )
                    food.color = current_color
                    xtest = random.randint(0, WIDTH // BLOCK_SIZE - 1)
                    ytest = random.randint(0, HEIGHT // BLOCK_SIZE - 1)
                    if xtest not in range( snake.body[0].x, snake.body[-1].x):
                        for i in range(0, len(current_level.walls)):
                            if current_level.walls[i].x != xtest:
                                if ytest not in range( snake.body[0].y, snake.body[-1].y):
                                    for i in range(0, len(current_level.walls)):
                                        if current_level.walls[i].y != ytest:
                                            condition = False
                food.location.x = xtest
                food.location.y = ytest   
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    dx, dy = 0, -1
                elif event.key == pygame.K_DOWN:
                    dx, dy = 0, +1
                elif event.key == pygame.K_RIGHT:
                    dx, dy = 1, 0
                elif event.key == pygame.K_LEFT:
                    dx, dy = -1, 0

        scores = font_small.render(str(SCORE), True, RED)
        level_tab = font_small.render('level: ' + str(level), True, RED)

        current_level.draw()
        snake.move(dx, dy)
        if snake.check_collision(food):
            if food.color == GREEN:
                SCORE += 1 
                snake.body.append(
                    Point(snake.body[-1].x, snake.body[-1].y)
                )

            if food.color == FORESTGREEN:
                SCORE += 2 
                snake.body.append(
                    Point(snake.body[-1].x, snake.body[-1].y)
                )
                snake.body.append(
                    Point(snake.body[-1].x, snake.body[-1].y)
                )               
            pygame.time.set_timer(FOOD_SPAWNER, 0)
            pygame.time.set_timer(FOOD_SPAWNER, 8000)

            current_color = random.choice( [GREEN, FORESTGREEN])
            food.color = current_color
            condition = True
            while condition:
                xtest = random.randint(0, WIDTH // BLOCK_SIZE - 1)
                ytest = random.randint(0, HEIGHT // BLOCK_SIZE - 1)
                if xtest not in range( snake.body[0].x, snake.body[-1].x):
                    for i in range(0, len(current_level.walls)):
                        if current_level.walls[i].x != xtest:
                            if ytest not in range( snake.body[0].y, snake.body[-1].y):
                                for i in range(0, len(current_level.walls)):
                                    if current_level.walls[i].y != ytest:
                                        condition = False
            food.location.x = xtest
            food.location.y = ytest   
        if snake.check_collision_wall( current_level):
            time.sleep( 0.5 )
            SCREEN.fill(RED)
            SCREEN.blit(game_over, (7 * BLOCK_SIZE, 10 * BLOCK_SIZE))
            pygame.display.update()
            time.sleep(2)
            pygame.quit()

        snake.draw()
        food.draw()
        draw_grid()
        SCREEN.blit(scores, (10,10))
        SCREEN.blit(level_tab, (10,50))
        pygame.display.flip()
        clock.tick(SPEED)


if __name__ == '__main__':
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

---

# 3. Paint

In [1]:
def main():
    pygame.init()
    screen = pygame.display.set_mode((640, 480))
    clock = pygame.time.Clock()
    
    radius = 15
    x = 0
    y = 0
    mode = 'blue'
    points = []
    painting_mode = 'circle'
    
    while True:
        
        pressed = pygame.key.get_pressed()
        
        alt_held = pressed[pygame.K_LALT] or pressed[pygame.K_RALT]
        ctrl_held = pressed[pygame.K_LCTRL] or pressed[pygame.K_RCTRL]
        
        for event in pygame.event.get():
            
            # Determine if X was clicked, or Ctrl+W or Alt+F4 was used
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                return
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w and ctrl_held:
                    return
                if event.key == pygame.K_F4 and alt_held:
                    return
                if event.key == pygame.K_ESCAPE:
                    return
            
                # Determine if a letter key was pressed for color and shape modes
                if event.key == pygame.K_r:
                    mode = 'red'
                elif event.key == pygame.K_g:
                    mode = 'green'
                elif event.key == pygame.K_b:
                    mode = 'blue'
                elif event.key == pygame.K_l:
                    painting_mode = 'rect'
                elif event.key == pygame.K_c:
                    painting_mode = 'circle'
                elif event.key == pygame.K_e:
                    painting_mode = 'eraser'
                elif event.key == pygame.K_1:
                    painting_mode = 'equilateral_triangle'
                elif event.key == pygame.K_2:
                    painting_mode = 'right_triangle'
                elif event.key == pygame.K_3:
                    painting_mode = 'rhombus'

            
            if event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:  # Left click grows radius
                    radius = min(200, radius + 1)
                elif event.button == 3:  # Right click shrinks radius
                    radius = max(1, radius - 1)
            
            if event.type == pygame.MOUSEMOTION:
                # If mouse moved, add point to list
                position = event.pos
                points = points + [position]
                points = points[-256:]
                
        screen.fill((0, 0, 0))
        
        # Draw all points
        i = 0
        while i < len(points) - 1:
            drawLineBetween(screen, i, points[i], points[i + 1], radius, mode, painting_mode)
            i += 1
        
        pygame.display.flip()
        
        clock.tick(60)

def drawLineBetween(screen, index, start, end, width, color_mode, painting_mode):
    c1 = max(0, min(255, 2 * index - 256))
    c2 = max(0, min(255, 2 * index))
    
    if color_mode == 'blue':
        color = (c1, c1, c2)
    elif color_mode == 'red':
        color = (c2, c1, c1)
    elif color_mode == 'green':
        color = (c1, c2, c1)
    
    dx = start[0] - end[0]
    dy = start[1] - end[1]
    iterations = max(abs(dx), abs(dy))
    
    for i in range(iterations):
        progress = 1.0 * i / iterations
        aprogress = 1 - progress
        x = int(aprogress * start[0] + progress * end[0])
        y = int(aprogress * start[1] + progress * end[1])
        
        # Draw shapes based on the painting mode
        if painting_mode == 'circle':
            pygame.draw.circle(screen, color, (x, y), width)
        elif painting_mode == 'rect':
            pygame.draw.rect(screen, color, (x, y, width + 30, width + 30))
        elif painting_mode == 'equilateral_triangle':
            pygame.draw.polygon(screen, color, [(x - width, y), (x, y - width), (x + width, y)])
        elif painting_mode == 'right_triangle':
            pygame.draw.polygon(screen, color, [(x, y), (x, y - width), (x + width, y)])
        elif painting_mode == 'rhombus':
            pygame.draw.polygon(screen, color, [(x - width, y), (x, y - width), (x + width, y), (x, y + width)])
        elif painting_mode == 'eraser':
            pygame.draw.rect(screen, (0, 0, 0), (x - width // 2, y - width // 2, width, width))

main()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'sys' is not defined